# WestCoastAD Optimization Package

# 1. Introduction 

Optimization theories and methods have been applied in many fields to solve various practical problems.
Given the advances in computational systems, optimization techniques have become increasingly important and popular in different engineering applications. 
Optimization is one of the core content of machine learning. 
The essence of most machine learning algorithms is to build an optimization model and learn the optimal parameters of the objective function from the given data.


Unconstrained scalar function optimization problems can be solved exactly by setting the derivative to zero and solving the resulting equation. 
However, we can't always solve the resulting equation especially when dealing with more complex higher-dimensional problems 
because directly computing the solution is hard and often impossible. 
Thus, we have to use iterative optimization methods such as gradient descent, 
which heavily relies on calculating the derivative of the objective function. 
Gradient plays an essential role in machine learning.
Neural networks can gradually increase accuracy with every training session through the process of gradient descent (Wang, 2019). 
However, for many problems, calculating derivatives analytically is both time-consuming and computationally impractical. 
The ability to efficiently evaluate derivatives is essential in optimization systems.

There are three main methods of calculating derivatives: 
numerical differentiation, symbolic differentiation, and automatic differentiation. 
Specifically, symbolic differentiation involves computing exact expressions for the derivatives in terms of 
the function variables by representing the expression as a tree and manipulating it using the rules of differentiation. 
The derivatives can quickly become very complicated for complex functions and higher-order derivatives. 
Thus, for complicated functions, symbolic differentiation is not easily generalizable and becomes computationally inefficient 
due to redundant evaluations of different components of the function. Numerical differentiation aims to approximate derivatives 
through finite differentiating, but the solution accuracy is greatly affected by the truncation and round-off errors 
associated with finite difference formulas.

Automatic differentiation (AD) is a set of techniques to numerically evaluate the derivative of a function specified by a computer program. 
It can give exact answers in constant time(Saroufim, 2019), 
which is a powerful tool to automate the calculation of derivatives, 
especially when differentiating complex algorithms and mathematical functions(Margossian, 2019). 
Automatic differentiation is more computationally efficient and more generalizable than 
symbolic differentiation and can find exact derivatives as opposed to numerical differentiation. 

In WestCoastAD Optimization Package, we implement several common optimization methods with a 
forward mode automatic differentiation core to compute gradients. 
In Section 2, we describe the mathematical background of some common optimization methods and the concepts of automatic 
differentiation. In Section 3, you can find the installation instruction and usage demos. 
In Sections 4 to 5, we describe the  organization and implementation of classes and functiones in WestCoastAD. 
We then discuss the broader impact and inclusicity statement in Section 6 and lastly our future plan to 
extend the package in Section 7.


# 2. Background

## 2.1 Optimization Algorithms 

There are various iterative optimization algorithms. All these algorithms start with an initial guess for the optimal inputs to the optimization function and iteratively update this guess by a small amount using some type of information about that function. Once a certain stopping condition is met (for instance a minimum value constraint on the update size or a maximum constraint on the number of iterations), the algorithm will terminate and return its last estimate of the optimal inputs to the function and the optimal value of the function. 

While these algorithms can converge to the optima of convex functions, they are not guaranteed to find the global optima of non-convex functions and are susceptible to convergence to local optima based on their initialization. Despite this, iterative optimization functions are ubiquitous in machine learning as finding local optima can still be useful. Below, we will go over various optimization algorithms that have been implemented in WestCoastAD. In the context of WestCoastAD, an optimization problem refers to the minimization of a function so we will be using these two terms interchangeably.

### 2.1.1 Gradient Descent

Optimization algorithms that belong to this category iteratively update the function inputs by subtracting the gradient of the optimization function with respect to these variables at the current estimate for the variables. This is because mathematically, the negative of the gradient points in the direction of steepest descent. Thus by greedily moving in the steepest local direction, we are guaranteed to improve our estimations at every iteration. 

$$
x_{t+1} = x_{t} - \alpha \nabla f(x_{t})
$$

The $\alpha$ parameter above is referred to as the learning rate of the algorithm and specifies how large a step the algorithm should take in each iteration.

Several variations of the gradient descent update have been proposed in order to help improve its convergence and the speed of the algorithm.

#### 2.1.1.1 Momentum Gradient Descent

While the gradient locally points in the direction of steepest descent, it does not necessarily point along the shortest path to the function's minimum. Often times gradient descent will oscillate around the shortest path. Momentum gradient descent helps speed up the algorithm by using an exponential moving average of the gradients at each iteration thus making the updates better align with the shortest path to the minimum:

$$
x_{t+1} = x_{t} - \alpha m_t
$$

$$
m_t = \beta m_{t-1} + (1-\beta) \nabla f(x_{t})
$$

Where $\beta$ is a user specified parameter commonly set to 0.9.

#### 2.1.1.2 AdaGrad

As gradient descent gets closer to a minimum, taking smaller steps can help us converge closer to the minimum. However, if we set the learning rate too small from the beginning, the algorithm will take too long to converge. AdaGrad gradually decreases the learning rate over time to help with convergence without slowing down the algorithm.

$$
x_{t+1} = x_{t} - \frac{\alpha}{\sqrt{v_t + \epsilon}} m_t
$$

$$
v_t = v_{t-1} + (\nabla f(x_{t}))^2
$$

Where $\epsilon$ is a small number added to avoid division by zero.

#### 2.1.1.3 RMSprop

Similar to AdaGrad, this variation of gradient descent also tries to achieve a gradual decrease in the learning rate over time to help with convergence.

$$
x_{t+1} = x_{t} - \frac{\alpha}{\sqrt{v_t + \epsilon}} m_t
$$

$$
v_t = \beta v_{t-1} + (1- \beta) (\nabla f(x_{t}))^2
$$

Where $\epsilon$ is a small number added to avoid division by zero and $\beta$ is a user specified parameter commonly set to 0.9.

#### 2.1.1.4 Adam

Adam combines RMSprop and Momentum gradient descent in order to achieve the desireable traits of both variations.

$$
x_{t+1} = x_{t} - \frac{\alpha}{\sqrt{v_t} + \epsilon} m_t
$$

$$
m_t = \frac{n_t}{1- \beta_1^t}
$$

$$
v_t = \frac{w_t}{1- \beta_2^t}
$$

$$
n_t = \beta_1 n_{t-1} + (1- \beta_1) (\nabla f(x_{t}))^2
$$

$$
w_t = \beta_2 w_{t-1} + (1- \beta_2) (\nabla f(x_{t}))^2
$$

Where $\epsilon$ is a small number added to avoid division by zero and $\beta_1 = 0.9$ and $\beta_2=0.999$.

### 2.1.2 Quasi-Newton Methods

Newton's method of optimization involves the computation of gradients and hessians. Hessians are very expensive to compute for functions with several input variables. Quasi-Newton methods forego hessian computations by starting with an initial guess for the Hessian and using gradient information to update this approximation at every iteration along with the optimal variable estimates. Thus Quasi-Newton algorithms complete the following steps in every iteration:

1. Solve $B_t \Delta x_{t} = - \nabla f(x_t)$

2. Update $x_{t+1}$ with $\Delta x_{t}$

3. Update $B_{t+1}$ based on $B_t$

Where $B$ is an approximation of the hessian. Different Quasi-Newton methods vary in how they update $B$.

#### 2.1.2.1 BFGS

BFGS uses a rank two update in step 3 which is given by: 

$$
B_{t+1}^{-1} = \left( I - \frac{ s_t y_t^T }{ y_t^T s_t } \right) B_t^{-1} \left( I - \frac{ y_t s_t^T }{ y_t^T s_t } \right) + \frac{ s_t s_t^T }{ y_t^T s_t }
$$

Where $s_t = \alpha \Delta x_{t}$ and $y_t = \nabla f(x_{t+1}) - \nabla f(x_t)$

## 2.2 Automatic Differentiation

As can be seen, all the aforementioned methods require derivative computations. Automatic differentiation is an efficient way of computing these derivatives which can be used with various complex functions. Automatic differentiation assumes that we are working with a differentiable function composed of a finite number of elementary functions and operations with known symbolic derivatives. The table below shows some examples of elementary functions and their respective derivatives:

| Elementary Function    | Derivative              |
| :--------------------: | :----------------------:|
| $x^3$                  | $3 x^{2}$               | 
| $e^x$                  | $e^x$                   |
| $\sin(x)$              | $\cos(x)$               |
| $\ln(x)$               | $\frac{1}{x}$           |

Given a list of elementary functions and their corresponding derivatives, the automatic differentiation process involves the evaluations of the derivatives of complex compositions of these elementary functions through repeated applications of the chain rule:

$$
 \frac{\partial}{\partial x}\left[f_1 \circ (f_2 \circ \ldots (f_{n-1} \circ f_n)) \right] = 
\frac{\partial f_1}{\partial f_2} \frac{\partial f_2}{\partial f_3} \ldots \frac{\partial f_{n-1}}{\partial f_n}\frac{\partial f_n}{\partial x}
$$

This process can be applied to the evaluation of partial derivatives as well thus allowing for computing derivatives of multivariate and vector-valued functions.

### 2.2.1 Computational Graph

The forward mode automatic differentiation process described above can be visualized in a computational graph, a directed graph with each node corresponding to the result of an elementary operation.

For example, consider the simple problem of evaluating the following function and its derivative at $x=2$:
$$
f(x) = x^3 +2x^2
$$
The evaluation of this function can be represented by the evaluation trace and computational graph below where the numbers in orange are the function values and the numbers in blue are the derivatives evaluated after applying each elementary operation:

<img src="https://raw.githubusercontent.com/anita76/playground/master/src/ex_comp_graph.png" width="75%" />

| Trace       | Elementary Function     | Value   | Derivative                | Derivative Value      |
| :---------: | :----------------------:| :------:| :------------------------:| :--------------------:|
| $v_1$       | $x$                     | $2$     | $\dot{x}$                 | $1$                   |
| $v_2$       | $v_1^2$                 | $4$     | $2v_1 \dot{v}_1$          | $4$                   |
| $v_3$       | $v_1^3$                 | $8$     | $3v_1^2 \dot{v}_1$        | $12$                  |
| $v_4$       | $2v_2$                  | $8$     | $2 \dot{v}_2$             | $8$                   |
| $v_5$       | $v_3 + v_4$             | $16$    | $ \dot{v}_3 + \dot{v}_4$  | $20$                  |



# 3. How to Use WestCoastAD

### 3.1 Installation with GitHub

1. Clone the package repository:

    `git clone https://github.com/West-Coast-Differentiators/cs107-FinalProject.git`


2. Create and activate a new environment:
    
    * Method 1: (Note: If you do not have Anaconda installed, follow the instructions on their [website](https://deepnote.com/project/07e48b43-3d96-4960-83bf-911527fb55ea?cellId=00002-de6540a6-d222-4d8b-b063-a392b8a9b7d4#/milestone2-2.ipynb) to install it.)
 
        * Create a new conda environment:
        
        `conda create --name <env_name> python=3.8` 

        * Activate your virtual environment:

        `conda activate <env_name>`
    
    * Method 2: 
        * Install pip environment package

        `pip install virtualenv` 
    
        * Create a new pip environment:
        
        `virtualenv <env_name>`

        * Activate your virtual environment:

        Mac OS / Linux: `source env_name/bin/activate`

        Windows: `env_name\Scripts\activate`

3. Navigate to the base repository:

    `cd cs107-FinalProject`

4. Install the package and its requirements:

    `pip install ./`

5. You may check that the installation was successful by running the package tests:

    `python -m unittest discover -s WestCoastAD/test -p '*_test.py'`

6. To deactivate the virtual environment

    `conda deactivate` (or `deactivate` for pip virtualenv)

### 3.2 Installation with pip

1. Create and activate a new environment:
    * Method 1: (Note: If you do not have Anaconda installed, follow the instructions on their [website](https://deepnote.com/project/07e48b43-3d96-4960-83bf-911527fb55ea?cellId=00002-de6540a6-d222-4d8b-b063-a392b8a9b7d4#/milestone2-2.ipynb) to install it.)
    
        * Create a new conda environment:
        
        `conda create --name <env_name> python=3.8` 

        * Activate your virtual environment:

        `conda activate <env_name>`
    
    * Method 2: 
        * Install pip environment package

        `pip install virtualenv` 
    
        * Create a new pip environment:
        
        `virtualenv <env_name>`

        * Activate your virtual environment:

        Mac OS / Linux: `source env_name/bin/activate`

        Windows: `env_name\Scripts\activate`
        
2. Install the package and its requirements:

    `pip install westcoastAD`

3. To deactivate the virtual environment

    `conda deactivate` (or `deactivate` for pip virtualenv)


### 3.3 Demos

#### 3.3.1 Optimization 

##### 3.3.1.1 Minimizing a univariate scalar function:

In [ ]:
# import all the necessary libraries
>>> import numpy as np
>>> import WestCoastAD as ad

# define the function that you wish to optimize. Your function can only include elementary
# operations that are supported by WestCoastAD. Please refer to the Software Implementation
# section of this documentation for a complete list.

>>> def my_objective_f(x):
>>>    return x*x.log(base=2)

# create an optimizer class with your objective function and an array of initial values for
# the inputs to the function that you want to optimize.

# You may use one of the initializer classes provided by WestCoastAD
>>> initialization = ad.RandomNormal(mean=1,stddev=0.2)(shape=1) 

# Or initialize the values yourself by creating a 1D numpy array of the right size
>>> initialization = np.array([2])

>>> optimizer = ad.Optimizer(my_objective_f, initialization)

# You can call any of the optimization algorithms provided by WestCoastAD on your optimizer
# You may need to try different parameter settings in order for the algorithm to converge.
# The first returned value is the best value of the function and the second value is an array
# containing the best values for the inputs to the function.
>>> optimizer.gd_optimize(num_iterations=1000, learning_rate=0.01)
(-0.530737845423043, array([0.36787944]))

>>> optimizer.adam_optimize(num_iterations=1000, learning_rate=0.01)
(-0.530737845423043, array([0.36787944]))

# you may access the history of the optimization function values to check for convergence.
# The history will include the progression of the optimization function values over the
# algorithm iterations for the last optimization algorithm that you ran.
history = optimizer.val_history


 ##### 3.3.1.2 Minimizing a multivariate scalar function:

In [ ]:
>>> import numpy as np
>>> import WestCoastAD as ad

# You may define multivariate scalar functions as a function of multiple
# scalar inputs

>>> def my_objective_f1(x, y, z):
>>>    return (x-2)**2*y**2 + (1-z)**2

# Or you may define them as a function with a single vector input 
>>> def my_objective_f2(x):
>>>    return (x[0]-2)**2*x[1]**2 + (1-x[2])**2

# create an optimizer class with your objective function and an array of initial values for
# the inputs to the function that you want to optimize. The size of the initalization array
# has to match the size of the input vector or the number of inputs to your function if your
# functions takes scalar inputs.
>>> initialization = np.array([3, 5, -1])

# When creating an optimizer, you must set the scalar parameter to False if your
# objective function takes a vector as input.
>>> optimizer = ad.Optimizer(my_objective_f1, initialization)
>>> optimizer = ad.Optimizer(my_objective_f2, initialization, scalar=False)

>>> optimizer.bfgs_optimize(num_iterations=100000, learning_rate=0.001)
(6.460561661947661e-26, array([2.00000000e+00, 1.55279742e-05, 1.00000000e+00]))

#### 3.3.2 Automatic Differentiation

You may also use WestCoastAD to differentiate multivariate and univariate scalar and vector functions.

##### 3.3.2.1 Differentiating a univariate scalar function

In [ ]:
# import all the necessary libraries
>>> import numpy as np
>>> import WestCoastAD as ad

# define the function that you wish to differentiate. Your function can only include elementary
# operations that are supported by WestCoastAD. Please refer to the Software Implementation
# section of this documentation for a complete list.

>>> def my_func(x):
>>>    return x.sin() + x.cos()

# You can use WestCoastAD's differentiate function to evaluate your function value and derivative
# you must provide differentiate with a 1D array containing the values of the inputs to your function.
# The first returned value is the function value and the returned array contains the derivative value
>>> x_val = np.array([np.pi/2])
>>> ad.differentiate(my_func, variable_values=x_val)
(1.0, array([-1.]))

Alternatively, you can use WestCoastAD's Variable class to write the function that you want to differentiate. This allows you to experiment with different derivative seeds.

In [ ]:
>>> from WestCoastAD import Variable
>>> import numpy as np

# define the variables of your function as WestCoastAD variables with the value at which you 
# want to evaluate the derivative and  a derivative seed value
>>> x = Variable(value=np.pi/2, derivative_seed=1)

# define your function in terms of the WestCoastAD variable objects
>>> f = x.sin() + np.cos(x)
>>> print(f.value)
1.0
>>> print(f.derivative)
-0.9999999999

##### 3.3.2.2 Differentiating a multivariate scalar function:

In [ ]:
>>> import numpy as np
>>> import WestCoastAD as ad

# You may define multivariate scalar functions as a function of multiple
# scalar inputs

>>> def my_func1(x, y, z):
>>>    return (x-2)**2*y**2 + (1-z)**2

# Or you may define them as a function with a single vector input 
>>> def my_func2(x):
>>>    return (x[0]-2)**2*x[1]**2 + (1-x[2])**2

# The array below specifies the values x=3, y=0, and z=4 which will be given to the differentiate
# function
>>> variable_values = np.array([1, 0, 4])

# When calling the differentiate function, you must set the scalar parameter to False if your
# objective function takes a vector as input.
# The returned array contains the gradient with respect to x, y, and z in this order
>>> ad.differentiate(my_func1, variable_values)
(9, array([0., 0., 6.]))
>>> ad.differentiate(my_func2, variable_values, scalar=False)
(9, array([0., 0., 6.]))

Alternatively, you can use WestCoastAD's Variable class to write the function that you want to differentiate. This allows you to experiment with different derivative seeds.

In [ ]:
>>> from WestCoastAD import Variable, multivariate_dimension_check
>>> import numpy as np

# define the variables of your function as WestCoastAD variables with the value at which you 
# want to evaluate the derivative and  a derivative seed value. Your variables must have the
# derivative seeds with the same dimensionality. We've used a dimensionality of 3 in this
# example since there are 3 variables
>>> x = Variable(value=1, derivative_seed=np.array([1, 0, 0]))
>>> y = Variable(value=0, derivative_seed=np.array([0, 1, 0]))
>>> z = Variable(value=4, derivative_seed=np.array([0, 0, 1]))

# you may use WestCoastAD to check that your variable functions have equal dimensionalities
>>> multivariate_dimension_check([x, y, z])
True

# define your function in terms of the WestCzzzoastAD variable objects
>>> f = (x-2)**2*y**2 + (1-z)**2
>>> print(f.value)
9
>>> print(f.derivative)
[0., 0., 6.]

##### 3.3.2.3 Differentiating a univariate vector function:

In [1]:
>>> import numpy as np
>>> import WestCoastAD as ad

# To define a vector function, create a python function that returns the 
# vector function as a 1D numpy array

>>> def my_func(x):
>>>    return np.array([x*2, x, x.log(base=2)])

# Provide the function and its variable value to differentiate 
>>> value, derivative = ad.differentiate(my_func, variable_values=np.array([2]))

# The value will be returned as a 1D array.
>>> print(value)
[4. 2. 1.]

# The derivative will be returned as a 2D array.
>>> print(derivative)
[[2.        ]
 [1.        ]
 [0.72134752]]

Alternatively, you can use WestCoastAD's Variable class to write the function that you want to differentiate. This allows you to experiment with different derivative seeds.

In [ ]:
>>> from WestCoastAD import Variable, vector_function_jacobian, vector_function_value

# define the variable of your function as a WestCoastAD variable
>>> x = Variable(value=2, derivative_seed=1)

# define your function as a numpy array in terms of the WestCoastAD variable object
>>> f = np.array([x*2, x, x.log(base=2)])

# you can obtain the value of your vector function using vector_function_value
>>> vector_function_value(f)
array([4., 2., 1.])

# you can obtain the Jacobian of your vector function using vector_function_jacobian
>>> vector_function_jacobian(f)
array([[2.        ],
       [1.        ],
       [0.72134752]])

##### 3.3.2.4 Differentiating a multivariate vector function:

In [ ]:
>>> import numpy as np
>>> import WestCoastAD as ad

# To define a vector function, create a python function that returns the 
# vector function as a 1D numpy array

>>> def my_func1(x, y):
>>>    return np.array([x*y, x-y, x.log(base=2)])

# You may also define your vector function to take a single vector as input
>>> def my_func2(x):
>>>    return np.array([x[0]*x[1], x[0]-x[1], x[0].log(base=2)])

# Provide the function and its variable values to differentiate 
# The example below sets x=1 and y=3
>>> value, derivative = ad.differentiate(my_func1, variable_values=np.array([1, 3]))

# if your function takes a vector as input, you must set the scalar parameter to False
>>> value, derivative = ad.differentiate(my_func2, variable_values=np.array([1, 3]), scalar=False)

# The value will be returned as a 1D array.
>>> print(value)
[ 3. -2.  0.]

# The derivative will be returned as a 2D array. The value at (0, 0) is the derivative of
# f_0 with respect to x, the value at (0, 1) is the derivative of f_0 with respect to y,
# the value at (1, 0) is the derivative of f_1 with respect to x and so on.
>>> print(derivative)
[[ 3.          1.        ]
 [ 1.         -1.        ]
 [ 1.44269504  0.        ]]

Alternatively, you can use WestCoastAD's Variable class to write the function that you want to differentiate. This allows you to experiment with different derivative seeds.

In [ ]:
>>> from WestCoastAD import Variable, vector_function_jacobian, vector_function_value
>>> import numpy as np

# define the variables of your function as WestCoastAD variables
>>> x = Variable(value=1, derivative_seed=np.array([1, 0]))
>>> y = Variable(value=3, derivative_seed=np.array([0, 1]))

# define your function as a numpy array in terms of the WestCoastAD variable objects
>>> f = np.array([x*y, x-y, x.log(base=2)])

# you can obtain the value of your vector function using vector_function_value
>>> vector_function_value(f)
array([ 3., -2.,  0.])

# you can obtain the Jacobian of your vector function using vector_function_jacobian
>>> vector_function_jacobian(f)
array([[ 3.        ,  1.        ],
       [ 1.        , -1.        ],
       [ 1.44269504,  0.        ]])

# 4. Software Organization
## 4.1 Directory Structure
```bash
cs107-FinalProject  
             setup.py
             .travis.yml 
             README.MD
             .gitignore
             docs/
                             documentation.ipynb
                             milestone1.ipynb
                             milestone2.ipynb   
                             milestone2A.ipynb
                             milestone2_progress.ipynb
                             
             WestCoastAD/  
                             __init__.py
                             src/    
                                     __init__.py
                                     forward_mode_AD.py
                                     forward_mode_AD_auxiliaries.py
                                     initializer.py
                                     optimizer.py
                                     
                             tests/ 
                                     __init__.py
                                     forward_mode_AD_test.py 
                                     forward_mode_AD_auxiliaries_test.py
                                     initializer_test.py
                                     optimizer_test.py
                                     
                     
```
## 4.2 Modules and Basic Functionality

#### Docs Module
* This module contains the documentation for the package. The documentation include the information about the package including the purpose, the background information about Optimization and Auto Differentiation, the organization of the software, how to use the package and how to install the package.

#### Setup Module
* This module contains configurations to allow users to install our package. Inside the setup.py file, we use the setuptools library for packaging the modules and installing the package dependencies. Our package requires that numpy be installed. We do not use a requirements.txt for this library; all the dependencies are handled through setup.py.

#### Forward Mode Module
* This module includes the implementation of the forward mode of automatic differentiation. The module defines a Variable class that will take the value to be evaluated and the derivative seed value. The derivative and value of functions defined in terms of Variable objects can be accessed by viewing the `derivative` and `value` attributes of this class. Each method in the Variable class, contains extensive documentation about the function and doctests that provide examples. 

#### Optimization Module
* This is the extension module that was included in the final release of our package. As we developed the final release, the optimization feature become the main feature of our software. At the core of our optimization functions was automatic differentiation which allowed us to create various algorithms that are prevalent in machine learning applications. For each method in the Optimizer class, we have provided documentation about the function and doctests that provide examples. The software implementation section of the documentation below will describe each implementation in detail.

#### Forward Mode AD Auxiliaries Module
* This module was created to expand AD to handle vector functions with multiple real scalar or vector inputs. The module contains several methods that expand the functionality of the Variable class. A detailed explanation of the methods is in the software implementation section. All functions have extension documentation and doctest that provide examples.

#### Initializer Module
* This module was created as a way to help user create the required parameters for the various optimization methods. A detailed explanation of the methods is in the extension section. All functions have documentation and doctests that provide examples. 

#### Test Modules
* This module contains all the tests of our implementation including unit tests and acceptance tests. We used Python's unittest to define the tests and checked our coverage with CodeCov. To have access to the test modules, the github repo has to be cloned and made available locally. Following the instruction in the "How to Use WestCoastAD" section of this documentation, the tests can be run using the following command:
`python -m unittest discover -s WestCoastAD/test -p '*_test.py'`. Alternatively, you can run individual test files using: `python WestCoastAD/test/<TEST_FILE_NAME>.py`

## 4.3 Packaging and Build Pipeline

#### Packaging
* We used sdist and bdist_wheel to create distribution archives. Most modern pip versions use bdist_wheel but we included an sdist archive as a fall back. These were uploaded to PYPI with the use of Twine. Within our setup.py, we used setuptools following the standard python packaging protocols as described in the following tutorial: [https://python-packaging-tutorial.readthedocs.io/en/latest/setup_py.html](https://python-packaging-tutorial.readthedocs.io/en/latest/setup_py.html)

#### Distribution
* WestCoastAD is currently distributed on Github and the package has been uploaded to the package index PyPI. In the "How to Use WestCoastAD" section of this document, there is information about how to install a virtual environment, how to install the package itself directly from Github or through a pip install and if installed via github where to get the package.

#### Travis CI
* Travis CI allows for automatic building and testing of commits that are pushed to remote. It requires that all pull request be tested and if all the tests are passed, it will update the build automatically. Travis CI also contains a badge that will inform the user if the package is working or if it is down. The travis.yml file is located in our main directory.

#### CodeCov
* CodeCov provides details about our test coverage. This allows us to identify where tests need to be added and will help us discover possible bugs in the code. CodeCov also contains a badge that will inform the user of the percentage code coverage of our tests.




# 5. Software Implementation

This section covers the class and functions for `WestCoastAD`.

## 5.1 Automatic Differentiation

`Variable` is the base class used to instantiate gradient objects and to perform derivatives.
The instances of the `Variable` class should be used to express the scalar or vector function to be differentiated.

| Class Attributes        | Usage        |
| :----------------------- |:-------------|
| `value`                 | Value of the variable (`int` or `float`). The setter and getter for value are defined using the  `property` decorator.|
| `derivative`            | Derivative of the function (`int`, `float`, or 1D `numpy` array). A `seed derivative` can be provided at the time of instantiation. This attribute is set and retrieved using a `property` decorator.|

<br>

**Class Methods**

###### `__add__`
Dunder method for overloading the "+" operator. This method computes the value and the derivative of the summation operation on `int`, `float` & `Variable` instances. This method returns a new `Variable` object.

###### `__radd__`
Same method as `__add__` with reversed operands.

###### `__mul__`
Dunder method for overloading the "*" operator. This method computes the value and the derivative following the [product rule](https://en.wikipedia.org/wiki/Product_rule) on `int`, `float` & `Variable` instances. It returns a new `Variable` object.

###### `__rmul__`
Same method as `__mul__` with reversed operands.

###### `__sub__`
Dunder method for overloading the "-" operator. This method computes the value and the derivative of the substraction operation on `int`, `float` & `Variable` instances. It returns a new `Variable` object.

###### `__rsub__`
Same method as `__sub__` with reversed operands.

###### `__truediv__`
Dunder method for overloading the "/" operator. This method computes the value and the derivative following the [quotient rule](https://en.wikipedia.org/wiki/Quotient_rule) on `int`, `float` & `Variable` instances. It returns a new `Variable` object. This method throws a `ZeroDivisionError` if the divisor is 0.

###### `__rtruediv__`
Same method as `__truediv__` with reversed operands.

###### `__pow__`
Dunder method for overloading the "**" operator. This method computes the value and the derivative following the [power rule](https://en.wikipedia.org/wiki/Power_rule) if exponent is `int` or `float`.
It returns a new `Variable` object. This method throws a `ValueError` if:
* a negative valued variable/function is raised to a non-integer power (e.g. $x^{1.2}, x=-2$)
* a zero valued variable/function is raised to a non-positive power (e.g. $x^{-5}, x = 0$)
* a non-positive valued variable/function is raised to the power of another variable/function (e.g. $x^{sin(x)}, x=-1$). This is because the generalized power rule of differentiation takes the logarithm of the base and the logarithm is not defined for non-positive values.

###### `__rpow__`
Dunder method which handles the differentiation of equations of the form $a^{f(x)}$. It returns a new `Variable` object. This method throws a `ValueError` if:
* zero is raised to the power of a variable/function with a non-positive value (e.g. $0^{x}, x=0$). 
* a negative number is used as the base (e.g. $(-2)^x$)

###### `__neg__`
Dunder method for overloading the negation operation. This method computes the value and derivative of the negation operation and returns a new `Variable` object.

###### `__abs__`
Dunder method for overloading the abs function. This method computes the value and derivative of the absolute value of a variable and returns a `Variable` object. It will throw a `ValueError` if given a value of zero since the derivative of absolute value is not defined at zero.

###### `__repr__`
Dunder method for overloading the `repr` function. Returns the object representation of the class `Variable` in the form `Variable(value=value, derivative=derivative)`

###### `log`
This method computes the value and derivative of `log` function and returns a new `Variable` object. A `ValueError` is thrown for non-positive values.

###### `exp`
This method computes the value and derivative of `exp` function and returns a new `Variable` object

###### `sqrt`
Square root is a special case of pow and therefore calls `__pow__` with exponent `1\2`.

###### `sin`

This method computes the value and the derivative of the sine function. It returns a new `Variable` object.

###### `cos`

This method computes the value and the derivative of the cosine function. It returns a new `Variable` object.

###### `tan`

This method computes the value and the derivative of the tangent function. It returns a new `Variable` object. This method will throw a `ValueError` if given a value that is an odd multiple of $\frac{\pi}{2}$

###### `sinh`

This method computes the value and the derivative of the hyperbolic sine function. It returns a new `Variable` object.

###### `cosh`

This method computes the value and the derivative of the hyperbolic cosine function.  It returns a new `Variable` object.

###### `tanh`

This method computes the value and the derivative of the hyperbolic tangent function.  It returns a new `Variable` object.

###### `arcsin`

This method computes the value and the derivative of the arcsin function.  It should be given a value in  $[-1,1]$ for the derivative to be defined, otherwise a `ValueError` will be raised. It returns a new `Variable` object.

###### `arccos`

This method computes the value and the derivative of the arccos function. It should be given a value in  $[-1,1]$ for the derivative to be defined, otherwise a `ValueError` will be raised. It returns a new `Variable` object.

###### `arctan`

This method computes the value and the derivative of the arctan function. It returns a new `Variable` object.

###### `__abs__`

Dunder method for overloading the abs function. Computes the value and derivative of abs function

###### `__lt__`

Dunder method for overloading the less than comparison.
This operand will perform elementwise comparison of the value and derivative of self and other. It will return a tuple of booleans; the first element is True if the value of self is less than  the value of other and the second value is True if ALL the componenets of the derivative of self are (element-wise) less than the derivative of other. Thus this method can only be used to compare Variable objects of the same dimensionality.

###### `__le__`

Dunder method for overloading the less than or equal to comparison.
This operand will perform elementwise comparison of the value and derivative of self and other.
It will return a tuple of booleans; the first element is True if the value of self is less than or equal to the value of other and the second value is True if ALL the componenets of the derivative of self are (element-wise) less than or equal to the derivative of other. Thus this method can only be used to compare Variable objects of the same dimensionality.

###### `__eq__`

Dunder method for overloading the equal to comparison. This operand will perform elementwise comparison of the value and derivative of self and other.
It will return a tuple of booleans; the first element is True if the value of self is equal to the value of other and the second value is True if ALL the componenets of the derivative of self are (element-wise) equal to the derivative of other. Thus this method can only be used to compare Variable objects of the same dimensionality.

###### `__ne__`

Dunder method for overloading the not equal to comparison.
This operand will perform elementwise comparison of the value and derivative of self and other.
It will return a tuple of booleans; the first element is True if the value of self is not equal to the value of other and 
the second value is True if ANY components of the derivative of self are (element-wise) not equal to the derivative of other.
Thus this method can only be used to compare Variable objects of the same dimensionality.

###### `__gt__`

Dunder method for overloading the greater than comparison.
This operand will perform elementwise comparison of the value and derivative of self and other.
It will return a tuple of booleans; the first element is True if the value of self is greater than the value of other and 
the second value is True if ALL components of the derivative of self are (element-wise) greater than the derivative of other.
Thus this method can only be used to compare Variable objects of the same dimensionality.

###### `__ge__`

Dunder method for overloading the greater than or equal to comparison.
This operand will perform elementwise comparison of the value and derivative of self and other.
It will return a tuple of booleans; the first element is True if the value of self is greater than or equal to the value of other and 
the second value is True if ALL components of the derivative of self are (element-wise) greater than or equal to the derivative of other.
Thus this method can only be used to compare Variable objects of the same dimensionality.


###### `logistic`

This method computes the value and derivative of the standard logistic function of the form $1/(1 + e^{-x})$

#### Vector valued and multivariable functions 

WestcoastAD will also be able to work with functions that have a single vector (numpy array) as input. To achieve this capability, we have created a forward mode AD auxiliaries module. This module contains various method that can be used to compute values, derivatives, gradients, and Jacobians  of multi and univariate scalar and vector functions with scalar or vector inputs. These methods abstract out the creation of Variable objects thus making it easier for users to differentiate functions. Below is a description of each method.

- **`vector_function_jacobian`** - this function takes a function and its variable values as a inputs and returns the jacobian of the given function as a single numpy array. 
The input `scalar` should be set to False if the function is defined to take a single vector as input.

- **`vector_function_value`** - this function takes a function and its variable values as a inputs and returns the value of the given function as a single numpy array.
The input `scalar` should be set to False if the function is defined to take a single vector as input.

- **`multivariate_dimension_check`** - this function takes an numpy array of WestCoastAD variable instances and checks if the derivatives of the Variable objects have the same derivative dimensionality. If the dimensions are the same it will return True, else False.

- **`differentiate`** - This function can be used to differentiate scalar and vector functions of one or more variables. It takes the function and its variable values as inputs. 
The input `scalar` should be set to False if the function is defined to take a single vector as input.

#### External Dependencies

We use `numpy` to evaluate elementary functions which were not available as part of the Python
standard library (e.g. sin, cos, etc). We use it to initialize vector inputs for multivariate functions.
It is also used to compute gradients and Jacobians of multivariable and vector-valued functions.
`numpy.testing` is used for unit testing gradients. 



## 5.2 Optimization (our extension)

### `Initializer` Module  

This module was created to help the user initialize the Optimizer class. The module contains various classes that will allow the user to generate different variable initializations for the objective function that is passed to the Optimizer class. The classes are listed below and a description is provided. 

* `Initializer Class` 

This is the abstract base class, It contains the `__call__` dunder methods that will be overloaded in the other classes and a `get_config` method that is also overloaded in the subclasses. The `get_config` method when overloaded will display the parameters for that class. The `__call__` dunder method will return an array with the desired output for each class. This class cannot be used for initialization as it has no underlying implementation.

* `Zeros  Class` 

This will generate a numpy array of zeros. The user has to specify the size of the array and the class will generate an array of that size containing only zeros.

* `Ones Class` 

This will generate a numpy array of ones. The user has to specify the size of the array and the class will generate an array of that size containing only ones.

* `Constant Class` 

This will generate a numpy array of a specified value. The user will provide a constant value and the number of values that should be in the array. The user-provided information will be loaded into the get_config dictionary.

* `RandomUniform Class`

This will generate a numpy array of random values based on a uniform distribution. The user will enter a minimum value, maximum value and the number of values that should be in the array. The user-provided information will be loaded into the get_config dictionary.

* `RandomNormal Class` 

This will generate a numpy array of random values based on a normal distribution. The user will provide a mean, standard deviation and the number of values that should be in the array. The user-provided information will be loaded into the get_config dictionary.


### `Optimizer` Module

This class provides methods to perform optimization with gradient descent on objective functions built using the `Variable` class from `WestCoastAD`. `Optimizer` class has 6 different optimization algorithms.

| Class Attributes        | Usage        |
| :----------------------- |:-------------|
| `objective_function`                 | A python function that takes as input a single vector or one or more scalars and returns a 1D numpy array of functions if objective_function is a vector function or a single function if objective_function is a scalar function.The function should be expressed using instances of the `Variable` class.|
| `variable_initialization`            | a 1D numpy array of floats/ints containing initial values for the inputs to the objective function.|
| `scalar`             |A boolean which is `True` if the inputs to objective_function are one or more scalars otherwise False.|
| `verbose`            | A boolean specifying whether updates about the optimization process will be printed to the console.|
| `val_history`            | A list which captures history of the optimization function values to check for convergence|


<br>

**Class Methods**

**`gd_optimize(num_iterations=100, learning_rate=0.01, tolerance=None)`**

This method returns the minimum value of the objective function alongside the critical point 
using regular gradient descent algorithm.

*Parameters:*

- `num_iterations` - An int specifying maximum # of iterations of gradient descent.

- `learning_rate` - A float/int specifying the learning rate for gradient descent.

- `tolerance` - A float specifying the smallest tolerance for the updates to the variables. If the L2 norm of the update step is smaller than this value, gradient descent will terminate.

*Returns:*

- A tuple with minimum value of the objective function and a scalar/vector input values at which the minima occured.


**`momentum_optimize(num_iterations=100, learning_rate=0.01, beta=0.9, tolerance=None)`**

This method returns the minimum value of the objective function alongside the critical point using momentum based gradient descent learning.
Usage of momentum in gradient descent to minimize loss function is commonly used in machine learning. Our implementation uses the `derivative` attribute of the objective function written using `Variable` class to obtain gradients in each iteration.
An exponential moving average of the current and the past gradients are used to determine the minima.

*Parameters:*

- `num_iterations` - An int specifying maximum # of iterations of gradient descent.

- `learning_rate` - A float/int specifying the learning rate for gradient descent.

- `beta` - A float ranging between 0 and 1 specifying the sample weight for exponential average of weights. The recommended value for this parameter is 0.9

- `tolerance` - A float specifying the smallest tolerance for the updates to the variables. If the L2 norm of the update step is smaller than this value, gradient descent will terminate.

*Returns:*

- A tuple with minimum value of the objective function and a scalar/vector input values at which the minima occured.

**`adagrad_optimize(num_iterations=1000, learning_rate=0.01, epsilon=1e-7, tolerance=None)`**

This method returns the minimum value of the objective function alongside the critical point using adaptive gradient (Adagrad) algorithm. Adagrad adjusts the learning rate alpha by dividing it by the square root of the cumulative sum of current and past squared gradients. Please refer to the background section for more details.

*Parameters:*

- `num_iterations` - An int specifying maximum # of iterations of gradient descent.

- `learning_rate` - A float/int specifying the learning rate for gradient descent.

- `epsilon` - A float to prevent division by zero during optimization.

- `tolerance` - A float specifying the smallest tolerance for the updates to the variables. If the L2 norm of the update step is smaller than this value, gradient descent will terminate.

*Returns:*

- A tuple with minimum value of the objective function and a scalar/vector input values at which the minima occured.

**`rmsprop_optimize(num_iterations=1000, learning_rate=0.001, epsilon=1e-7, beta=0.9, tolerance=None)`**

This method performs RMSProp gradient descent optimization on the objective function (which is expressed using instances of the `Variable` class). This is an enhanced version of the Adagrad optimizers. It adjusts the learning rate alpha by dividing it by the exponential moving averages of gradients.

*Parameters:*

- `num_iterations` - An int specifying maximum # of iterations of gradient descent.

- `learning_rate` - A float/int specifying the learning rate for gradient descent.

- `epsilon` - A float to prevent division by zero during optimization.

- `beta` - A float ranging between 0 and 1 specifying the sample weight for exponential average of weights. The recommended value for this parameter is 0.9

- `tolerance` - A float specifying the smallest tolerance for the updates to the variables. If the L2 norm of the update step is smaller than this value, gradient descent will terminate.

*Returns:*

- A tuple with minimum value of the objective function and a scalar/vector input values at which the minima occured.

**`adam_optimize(num_iterations=1000, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, tolerance=None):`**

This method performs Adaptive Moment Estimation (ADAM) optimization on the objective function.

*Parameters:*

- `num_iterations` - An int specifying maximum # of iterations of gradient descent.

- `learning_rate` - A float/int specifying the learning rate for gradient descent.

- `beta1` - Exponential decay hyperparameter for the first moment estimates.

- `beta2` - Exponential decay hyperparameter for the second moment estimates.

- `epsilon` - A float to prevent division by zero during optimization.

- `tolerance` - A float specifying the smallest tolerance for the updates to the variables. If the L2 norm of the update step is smaller than this value, gradient descent will terminate.

*Returns:*

- A tuple with minimum value of the objective function and a scalar/vector input values at which the minima occured.

**`bfgs_optimize(num_iterations=1000, learning_rate=0.01, tolerance=None)`**

This method performs Quasi-Newton optimization of the objective function with BFGS updates.

*Parameters:*

- `num_iterations` - An int specifying maximum # of iterations of gradient descent.

- `learning_rate` - A float/int specifying the learning rate for gradient descent.

- `tolerance` - A float specifying the smallest tolerance for the updates to the variables. If the L2 norm of the update step is smaller than this value, gradient descent will terminate.

*Returns:*

- A tuple with minimum value of the objective function and a scalar/vector input values at which the minima occured.


# 6. Broader Impact and Inclusivity Statement

### 6.1 Broader Impact

Optimizers have a wide range of applications across various industries. Some of the use cases include 
cost/revenue optimization, optimal labor staffing at factories, automated location navigation, supply chain optimization, work allocation in crowd-sourcing markets.
`WestCoastAD` enables automatic optimization of real word problems, modeled as objective functions for optimal decision making.
There is more work to be done as elaborated in Futures Section but what we have currently is a step in the right direction.

Based on the usage context, optimization errors and lack of understanding of how optimizers work can affect societies in varying degrees.
For instance, inaccurate optimizations can result in delays/increased inventory costs in supply chain management. Knowledge about how an optimizer does optimization can have positive and negative impacts.
Per study elaborated in [6], lack of transparency in algorithmic management affected the morale of employees.
On the contrary, there were scenarios where knowledge of how the algorithm optimized the objective encouraged offenders to game the system.

It is important to design optimizers in `WestCoastAD` that converge faster with high accuracy. There is the potential misuse of showcasing these methods as methods that can find optimal results but in reality they are very sensitive to parameter settings and unless the optimization function is convex, there is no guarantee that the algorithm will be close to the true global optimum.
### 6.2 Software Inclusivity

We plan to release this software to the open source community for contribution.
The name `WestCoastAD` was something we came up with for the purposes of this class (since the core contributors were all on the Pacific time zone).
Prior to open sourcing, we will rename our package to make it location neutral.
We will also rename repo `master` branch to `main` per [Github guidelines](https://github.com/github/renaming).
We will hold the contributors and peer reviews accountable to a code review standard.
Google has published it's [engineering code review](https://github.com/google/eng-practices/blob/master/review/index.md) practice which we can adopt.
We welcome all members of the software community to contribute to this package. 
In addition to functionality extension, there are opportunities to localize the software to make it accessible for contributors across all regions.
We would translate the documentation to multiple languages. 


# 7. Future Plan

An automatic-differentiation-based optimization package can be applied in many fields to handle various practical problems. In our future extensions, we mainly focus on improving the efficiency of gradient computation in machine learning models.

### 7.1 Reverse mode/Backpropagation class
As we mentioned at the beginning, optimization packages are heavily used in deep learning. Training of a neural network is basically an optimization problem about finding the minimum of the loss function with respect to its set of weights. We will extend our class by adding reverse mode of automatic differentiation to accommodate back propagation. Reverse mode AD is more efficient at getting gradients of multivariate scalar functions of large dimensionality thus making it more suitable for neural networks. 

### 7.2 Common machine learning models class
Common machine learning models like linear regression and neural networks utilize optimization methods to train the model. We would like to extend our package to create classes for some common machine learning models in order to make it easier for users to utilize our optimizer for machine learning problems. The fit methods will be realized using our automatic-differentiation-based optimization class. 

### 7.3 Natural language processing models class
Natural language processing(NLP) is the main area of deep learning, with common tasks in syntactic analysis, machine translation, language modelling, and etc. However, training an NLP model usually is time-consuming. Having the ability to quickly train the models in NLP is very important. In Yu and Siskind(2013)'s work on sentence tracking, automatic differentiation based gradient descent is proved to save the training time dramatically in C language. We may extend our software to implement NLP models based on our AD backed gradient descent based optimizers. To achieve this, we will need to tackle efficiency issues. Most importantly, we need to support automatic differentiation for matrix and vector operations (e.g. vector norms, matrix multiplication) in order to avoid using for loops and delegate time consuming mathematical operations to lower level languages such as C as done in packages like numpy.


# 8. References
1.  Margossian, C. C. (2019). A review of automatic differentiation and its efficient implementation. Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, 9(4), 1–32. https://doi.org/10.1002/widm.1305

2. Saroufim, M. (2019, November 13). Automatic Differentiation Step by Step. Retrieved October 15, 2020, from https://marksaroufim.medium.com/automatic-differentiation-step-by-step-24240f97a6e6

3. Wang, C. (2019, March 3). Automatic Differentiation, Explained. Retrieved October15, 2020,from https://towardsdatascience.com/automatic-differentiation-explained-	b4ba8e60c2ad

4. Automatic Differentiation Background. Retrived October 20, 2020, from https://www.mathworks.com/help/deeplearning/ug/deep-learning-with-automatic-differentiation-in-matlab.html

5. Haonan Yu and Jeffrey Mark Siskind. Grounded language learning from video described with sentences. In Proceedings of the 51st Annual Meeting of the Association for Computational Linguistics, pages 53–63, Sofia, Bulgaria, 2013. Association for Computational
Linguistics.

6. Min Kyung Lee, Daniel Kusbit, Evan Metsky and Laura Dabbish. Working with Machines: The Impact of Algorithmic
and Data-Driven Management on Human Workers. Retrieved December11, 2020 from https://dl.acm.org/doi/10.1145/2702123.2702548